In [ ]:
### imports

# external modules
import os
import sys
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

from automasking.tools.automask_file_parsing import automask_to_map
import plotting.plottools as plottools
import tools.iotools as iotools
import tools.dftools as dftools

In [ ]:
# read automasks from json file

automask_data_dir = '../data/automask_data'
automask_data_file = os.path.join(automask_data_dir, f'automask_2024.json')

with open(automask_data_file, 'r') as f:
    automask_info = json.load(f)

In [ ]:
# get the number of automasked channels over time

num_masked_channels = {}
subsystems = ["BPix1", "BPix2", "BPix3", "BPix4"]
#subsystems = ["BPix2"]
for subsystem in subsystems:
    print(f'Calculating system {subsystem}...', end='\r')
    num_masked_channels[subsystem] = {key: np.sum(automask_to_map(automask_info[key][subsystem], subsystem=subsystem)) for key in automask_info.keys()}

In [ ]:
# make a plot of the above

# define colors
colors = {
    "BPix1": "mediumblue",
    "BPix2": "cornflowerblue",
    "BPix3": "blueviolet",
    "BPix4": "violet"
}

# as a function of time
xax = np.arange(len(num_masked_channels[subsystems[0]]))
fig,ax = plt.subplots(figsize=(18,6))

for subsystem in subsystems:
    values = np.array(list(num_masked_channels[subsystem].values()))
    ax.plot(xax, values, label=subsystem, color=colors[subsystem], alpha=0.5)
    
ax.set_xlabel('Automask timestamp (per minute)', fontsize=15)
ax.set_ylabel('Number of automasked channels', fontsize=15)
ax.set_yscale('log')
ax.grid(which='both')
ax.legend(fontsize=15)

# distribution
bins = np.linspace(-0.5, 100.5, num=102)
print(bins)
fig,ax = plt.subplots(figsize=(12,6))

for subsystem in subsystems:
    values = np.array(list(num_masked_channels[subsystem].values()))
    ax.hist(values, bins=bins, label=subsystem, color=colors[subsystem], histtype='step', linewidth=2)
    
ax.set_xlabel('Number of automasked channels', fontsize=15)
ax.set_ylabel('Number of automask timestamps', fontsize=15)
ax.set_yscale('log')
ax.grid(which='both')
ax.legend(fontsize=15)

In [ ]:
# find lumisections with a high number of masked channels

subsystem = 'BPix1'
threshold = 100

keys = np.array(list(num_masked_channels[subsystem].keys()))
values = np.array(list(num_masked_channels[subsystem].values()))
ids = np.nonzero(values > threshold)[0]
keys = keys[ids]
print(len(keys))
print(keys[:10])